# Imports 

In [71]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler
import math

from qiskit import execute, QuantumCircuit, Aer

# Classical Pre-processing

In [72]:
train_df = pd.read_csv('./Titanic_data/train.csv')
test_df = pd.read_csv('./Titanic_data/test.csv')
submission_df = pd.read_csv('./Titanic_data/gender_submission.csv')

In [73]:
display(train_df.head())
display(test_df.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# Drop unnecessary columns 

In [74]:
train_df.drop(columns=['Name', 'Ticket', 'Cabin'], inplace=True)
test_df.drop(columns=['Name', 'Ticket', 'Cabin'], inplace=True)

In [75]:
display(train_df.head())
display(test_df.head())

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S


# Drop NaNs

In [76]:
train_df.dropna(inplace=True)
test_df.fillna(0)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...,...
413,1305,3,male,0.0,0,0,8.0500,S
414,1306,1,female,39.0,0,0,108.9000,C
415,1307,3,male,38.5,0,0,7.2500,S
416,1308,3,male,0.0,0,0,8.0500,S


In [77]:
print(train_df.shape, test_df.shape)

(712, 9) (418, 8)


# One-hot encoding

In [78]:
one_hot = pd.get_dummies(train_df[["Pclass", "Sex", "Embarked"]].astype(str))
train_df = train_df.drop(columns=["Pclass", "Sex", "Embarked"])
train_df = train_df.join(one_hot)

display(train_df)

one_hot = pd.get_dummies(test_df[["Pclass", "Sex", "Embarked"]].astype(str))
test_df = test_df.drop(columns=["Pclass", "Sex", "Embarked"])
test_df = test_df.join(one_hot)

display(test_df)

,PassengerId,Survived,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,22.0,1,0,7.2500,0,0,1,0,1,0,0,1
1,2,1,38.0,1,0,71.2833,1,0,0,1,0,1,0,0
2,3,1,26.0,0,0,7.9250,0,0,1,1,0,0,0,1
3,4,1,35.0,1,0,53.1000,1,0,0,1,0,0,0,1
4,5,0,35.0,0,0,8.0500,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,886,0,39.0,0,5,29.1250,0,0,1,1,0,0,1,0
886,887,0,27.0,0,0,13.0000,0,1,0,0,1,0,0,1
887,888,1,19.0,0,0,30.0000,1,0,0,1,0,0,0,1
889,890,1,26.0,0,0,30.0000,1,0,0,0,1,1,0,0


,PassengerId,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,892,34.5,0,0,7.8292,0,0,1,0,1,0,1,0
1,893,47.0,1,0,7.0000,0,0,1,1,0,0,0,1
2,894,62.0,0,0,9.6875,0,1,0,0,1,0,1,0
3,895,27.0,0,0,8.6625,0,0,1,0,1,0,0,1
4,896,22.0,1,1,12.2875,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,0,0,8.0500,0,0,1,0,1,0,0,1
414,1306,39.0,0,0,108.9000,1,0,0,1,0,1,0,0
415,1307,38.5,0,0,7.2500,0,0,1,0,1,0,0,1
416,1308,NaN,0,0,8.0500,0,0,1,0,1,0,0,1


# Normalization

In [79]:
scaler = MinMaxScaler() 
arr_scaled = scaler.fit_transform(train_df[["Age", "SibSp", "Parch", "Fare"]]) 

df_scaled = pd.DataFrame(arr_scaled, columns=["Age", "SibSp", "Parch", "Fare"], index=train_df.index)
train_df = train_df.drop(columns=["Age", "SibSp", "Parch", "Fare"])
train_df = train_df.join(df_scaled)

display(train_df)

arr_scaled = scaler.fit_transform(test_df[["Age", "SibSp", "Parch", "Fare"]]) 

df_scaled = pd.DataFrame(arr_scaled, columns=["Age", "SibSp", "Parch", "Fare"], index=test_df.index)
test_df = test_df.drop(columns=["Age", "SibSp", "Parch", "Fare"])
test_df = test_df.join(df_scaled)

display(test_df)

,PassengerId,Survived,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Age,SibSp,Parch,Fare
0,1,0,0,0,1,0,1,0,0,1,0.271174,0.2,0.000000,0.014151
1,2,1,1,0,0,1,0,1,0,0,0.472229,0.2,0.000000,0.139136
2,3,1,0,0,1,1,0,0,0,1,0.321438,0.0,0.000000,0.015469
3,4,1,1,0,0,1,0,0,0,1,0.434531,0.2,0.000000,0.103644
4,5,0,0,0,1,0,1,0,0,1,0.434531,0.0,0.000000,0.015713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,886,0,0,0,1,1,0,0,1,0,0.484795,0.0,0.833333,0.056848
886,887,0,0,1,0,0,1,0,0,1,0.334004,0.0,0.000000,0.025374
887,888,1,1,0,0,1,0,0,0,1,0.233476,0.0,0.000000,0.058556
889,890,1,1,0,0,0,1,1,0,0,0.321438,0.0,0.000000,0.058556


,PassengerId,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Age,SibSp,Parch,Fare
0,892,0,0,1,0,1,0,1,0,0.452723,0.000,0.000000,0.015282
1,893,0,0,1,1,0,0,0,1,0.617566,0.125,0.000000,0.013663
2,894,0,1,0,0,1,0,1,0,0.815377,0.000,0.000000,0.018909
3,895,0,0,1,0,1,0,0,1,0.353818,0.000,0.000000,0.016908
4,896,0,0,1,1,0,0,0,1,0.287881,0.125,0.111111,0.023984
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,0,0,1,0,1,0,0,1,NaN,0.000,0.000000,0.015713
414,1306,1,0,0,1,0,1,0,0,0.512066,0.000,0.000000,0.212559
415,1307,0,0,1,0,1,0,0,1,0.505473,0.000,0.000000,0.014151
416,1308,0,0,1,0,1,0,0,1,NaN,0.000,0.000000,0.015713


# Correlation

In [80]:
weight_series = train_df.corr()["Survived"].drop(["Survived", "PassengerId"])

In [81]:
print(weight_series)

Pclass_1      0.297908
Pclass_2      0.086891
Pclass_3     -0.335364
Sex_female    0.536762
Sex_male     -0.536762
Embarked_C    0.195673
Embarked_Q   -0.048966
Embarked_S   -0.159015
Age          -0.082446
SibSp        -0.015523
Parch         0.095265
Fare          0.266100
Name: Survived, dtype: float64


# Calculate Theta for Test DataFrame

In [82]:
test_df['MU'] = test_df[["Pclass_1", "Pclass_2", "Pclass_3", "Sex_female", "Sex_male", "Embarked_C",
         "Embarked_Q", "Embarked_S", "Age", "SibSp", "Parch", "Fare"]].mul(weight_series.to_list()).sum(1)

display(test_df)

,PassengerId,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Age,SibSp,Parch,Fare,MU
0,892,0,0,1,0,1,0,1,0,0.452723,0.000,0.000000,0.015282,-0.954351
1,893,0,0,1,1,0,0,0,1,0.617566,0.125,0.000000,0.013663,-0.006838
2,894,0,1,0,0,1,0,1,0,0.815377,0.000,0.000000,0.018909,-0.561029
3,895,0,0,1,0,1,0,0,1,0.353818,0.000,0.000000,0.016908,-1.055813
4,896,0,0,1,1,0,0,0,1,0.287881,0.125,0.111111,0.023984,0.033674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,0,0,1,0,1,0,0,1,NaN,0.000,0.000000,0.015713,-1.026960
414,1306,1,0,0,1,0,1,0,0,0.512066,0.000,0.000000,0.212559,1.044687
415,1307,0,0,1,0,1,0,0,1,0.505473,0.000,0.000000,0.014151,-1.069050
416,1308,0,0,1,0,1,0,0,1,NaN,0.000,0.000000,0.015713,-1.026960


# Quantum Classifier

In [83]:
def pqc_classify(backend, passenger_state):
    qc = QuantumCircuit(1)
    qc.initialize(passenger_state, 0)
    qc.measure_all()
    count = execute(qc, backend).result().get_counts(qc)
    
    return int(list(map(lambda item : item[0], count.items()))[0])

In [84]:
def get_state(mu):
    theta = (1-mu) * math.pi
    return [math.cos(theta/2), math.sin(theta/2)]

In [85]:
submission_df = submission_df[0:0]

In [86]:
backend = Aer.get_backend('statevector_simulator')

for mu, passenger_id in zip(test_df["MU"], test_df["PassengerId"]):
    prediction = pqc_classify(backend, get_state(mu))
    
    append_list = [passenger_id, prediction]
    submission_df.loc[len(submission_df)] = append_list
    
    
display(submission_df)

,PassengerId,Survived
0,892,0
1,893,1
2,894,1
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,0
415,1307,0
416,1308,0


In [87]:
submission_df.to_csv('./Titanic_data/gender_submission.csv', index=False)